In [1]:
!pip install langchain_community -q
!pip install pypdf -q
!pip install langchain_google_genai -q
! pip install PyMuPDF -q
!pip install python-pptx -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 12.3 MB/s eta 0:00:00


In [2]:

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate,PromptTemplate

In [3]:
# TO JOIN ALL THE TEXT FROM ALL THE PAGES OF THE PDF DOCUMENTS

def doc_words_join(inp):
   loader = PyPDFLoader(inp)
   pages = loader.load_and_split()
   # print(len(pages)) #no of pages in the documents
   whole_doc_text = ''
   for i in range(len(pages)):
       whole_doc_text+=pages[i].page_content
   return whole_doc_text

In [4]:

allwords=doc_words_join('/content/2206784 Miraj Deep Bhandari.pdf')
print(allwords)

Module Code & Module Title 
CU6051NI Artificial Intelligence 
Assessment Weightage & Type 
25% Individual 
Semester 
2024 Autumn 
Project Name: Nepali Image Captioning 
Student Name: Miraj Deep Bhandari 
London Met ID: 22067814 
College ID: np01cp4a220197 
Group :L3C8 
Assignment Due Date: Wednesday, 25 December 2024 
Assignment Submission Date: Monday, 23 December 2024 
I confirm that I understand my coursework needs to be submitted online via Google Classroom 
under the relevant module page before the deadline in order for my assignment to be accepted 
and marked. I am fully aware that late submissions will be treated as non-submission and a mark 
of zero will be awarded.Table of Contents 
1) Introduction: .............................................................................................................. 1 
1.1 Aims: ..................................................................................................................... 2 
1.2 Objectives: .....................

In [5]:
## Splittting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
chunks = text_splitter.create_documents([allwords])

In [6]:
chunks_prompt="""
Please summarize the below Document. The summary must be very clear with proper phrases and easy words:
Document:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'], template=chunks_prompt)


In [13]:
system = """
You are tasked with generating a minimum of 20 multiple-choice questions (MCQs) based on the provided text. Each MCQ should be well-structured and follow the format below:

Question: [Clearly state the question in a concise and understandable manner.]
Options:
A) [Option 1]
B) [Option 2]
C) [Option 3]
D) [Option 4]
Correct Answer: [Specify the correct option]

Ensure that questions assess key concepts, facts, or inferences from the text. Options should be plausible and not overly obvious. Avoid ambiguous wording and ensure that each question has only one correct answer. Provide a mix of factual, conceptual, and application-based questions. Each question should be self-contained and understandable without additional context.

i wnat like this

**Question X:**
[Insert question here]

Options:
A) [Option A]
B) [Option B]
C) [Option C]
D) [Option D]

Correct Answer: [Correct option letter]


Note: Do not include any preamble. """

human="{text}" # yesma sab chunk ko summary combine vayera input hunxa     # yo ra mathi ko chunk prompt ma same hunparxa input field ko name


final_combine_prompt=ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template(system),
                                                 HumanMessagePromptTemplate.from_template(human)])


In [14]:
llm_model = GoogleGenerativeAI(model="gemini-pro", google_api_key="")

In [15]:

summary_chain = load_summarize_chain( llm=llm_model, chain_type='map_reduce', map_prompt=map_prompt_template, combine_prompt=final_combine_prompt,
                                         verbose=False)

output = summary_chain.invoke(chunks)


In [16]:
print(output['output_text'])

**Question 1:**
What is the primary goal of the image captioning system?
Options:
A) To generate accurate captions for images in Nepali
B) To improve the efficiency of image retrieval
C) To assist self-driving cars in navigation
D) To replace human image describers
Correct Answer: A

**Question 2:**
Which technique is used for image analysis in the system?
Options:
A) Recurrent Neural Networks
B) Vision Transformer
C) Convolutional Neural Networks
D) Generative Adversarial Networks
Correct Answer: B

**Question 3:**
What is the purpose of the GPT component in the system?
Options:
A) To extract features from the image
B) To generate captions based on the extracted features
C) To evaluate the accuracy of the generated captions
D) To store the image dataset
Correct Answer: B

**Question 4:**
Which metrics are likely used to evaluate the system's performance?
Options:
A) Caption accuracy and relevance
B) Image similarity score
C) Caption length
D) Execution time
Correct Answer: A

**Questi

In [17]:
import re

text = output['output_text']

def parse_questions(text):
    questions = {}
    question_blocks = re.split(r'\*\*Question (\d+):\*\*', text)[1:]  # Splitting on **Question X:** pattern

    for i in range(0, len(question_blocks), 2):
        q_number = int(question_blocks[i].strip())  # Extract question number
        q_content = question_blocks[i + 1].strip().split("\n")  # Extract question content

        question_text = q_content[0].strip()
        options = {}
        correct_answer = ""

        for line in q_content[1:]:
            match = re.match(r"([A-D])\) (.+)", line.strip())  # Match option pattern
            if match:
                options[match[1]] = match[2]
            elif line.startswith("Correct Answer:"):
                correct_answer = line.split(":")[1].strip()

        questions[q_number] = {
            "question": question_text,
            "options": options,
            "correct_answer": correct_answer
        }

    return questions

question_dict = parse_questions(text)
print(question_dict)


{1: {'question': 'What is the primary goal of the image captioning system?', 'options': {'A': 'To generate accurate captions for images in Nepali', 'B': 'To improve the efficiency of image retrieval', 'C': 'To assist self-driving cars in navigation', 'D': 'To replace human image describers'}, 'correct_answer': 'A'}, 2: {'question': 'Which technique is used for image analysis in the system?', 'options': {'A': 'Recurrent Neural Networks', 'B': 'Vision Transformer', 'C': 'Convolutional Neural Networks', 'D': 'Generative Adversarial Networks'}, 'correct_answer': 'B'}, 3: {'question': 'What is the purpose of the GPT component in the system?', 'options': {'A': 'To extract features from the image', 'B': 'To generate captions based on the extracted features', 'C': 'To evaluate the accuracy of the generated captions', 'D': 'To store the image dataset'}, 'correct_answer': 'B'}, 4: {'question': "Which metrics are likely used to evaluate the system's performance?", 'options': {'A': 'Caption accura